# 02. Feature & Target Dataset 구축

## 🎯 이 단계의 단 하나의 목적

> **Raw 데이터를 "모델과 운영이 공통으로 사용하는 단일 정식 데이터셋"으로 변환**

## 📋 책임 (Responsibility)

### ✅ 이 단계가 하는 것
1. **Feature Builder**: 기술적 지표 계산 (MA, RSI, MACD, Bollinger 등)
2. **Universe Meta Builder**: 운영 판단용 메타 지표 (유동성, 리스크 등)
3. **History Filtering**: Feature 준비 기간(60일) 이후 데이터만 유지
4. **스키마 검증**: 필수 컬럼 존재 확인
5. **Parquet 저장**: 단일 통합 데이터셋 생성

### ❌ 이 단계가 하지 않는 것
- **Target 생성** (→ 03단계에서 필요 시 생성)
- Train/Test Split (→ 03단계)
- Normalization/Scaling (→ 03단계)
- 샘플 제거/필터링 (→ 03단계)

## 🔄 데이터 흐름

```
data/01_raw/*.csv
    ↓ [Load]
  Raw DataFrame
    ↓ [build_features]
  + 기술지표
    ↓ [build_universe_meta]
  + 메타 지표
    ↓ [filter_by_history]
  Feature 준비 기간(60일) 제거
    ↓ [validate_schema]
  검증 통과
    ↓ [save]
data/03_processed/dataset.parquet
```

## 🔧 Setup

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm

# 기술적 지표 모듈 임포트
from src.features.technical import (
    calc_rsi,
    calc_macd,
    calc_bollinger,
    calc_sma,
    calc_volume_ratio
)

print("✅ Technical indicators module loaded")

✅ Technical indicators module loaded


In [2]:
# ==================== 설정 ====================

REFERENCE_DATE = "20251226"  # 종목 리스트 기준일
RAW_DATA_DIR = Path(f"data/01_raw/krx_prices_{REFERENCE_DATE}")
OUTPUT_DIR = Path("data/03_processed")
OUTPUT_FILE = OUTPUT_DIR / f"dataset_{REFERENCE_DATE}.parquet"

# Feature 준비 기간 (최장 rolling window)
MIN_HISTORY = 60  # MA_60이 가장 긴 기간

# 디렉토리 생성
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"📥 입력: {RAW_DATA_DIR}")
print(f"📤 출력: {OUTPUT_FILE}")
print(f"⏱️  최소 히스토리: {MIN_HISTORY}일 (Feature 준비 기간)")

📥 입력: data\01_raw\krx_prices_20251226
📤 출력: data\03_processed\dataset_20251226.parquet
⏱️  최소 히스토리: 60일 (Feature 준비 기간)


## 📦 Helper Functions

### 트랙 B 개념 적용: 책임 분리

In [3]:
# ==========================================
# Feature Builder: 기술적 지표 계산
# ==========================================

def build_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    기술적 지표 Feature 생성
    
    책임
    ----
    - 가격 기반 기술지표만 계산
    - 과거 정보만 사용 (미래 정보 절대 금지)
    - 종목별(ticker) 그룹 연산
    
    Parameters
    ----------
    df : DataFrame
        Raw 데이터 (최소: date, ticker, close, volume)
        
    Returns
    -------
    DataFrame
        원본 + feature_* 컬럼 추가
        
    Note
    ----
    기술적 지표 계산은 src.features.technical 모듈 사용
    """
    print("\n🔨 Building Features...")
    
    df = df.copy()
    df = df.sort_values(['ticker', 'date']).reset_index(drop=True)
    
    # 종목별 그룹 연산
    print("  - 이동평균 (MA 5, 20, 60)...")
    for window in [5, 20, 60]:
        df[f'feature_ma_{window}'] = df.groupby('ticker')['close'].transform(
            lambda x: calc_sma(x, window)
        )
    
    print("  - 변동성 (20일 표준편차)...")
    df['feature_volatility_20'] = df.groupby('ticker')['close'].transform(
        lambda x: x.pct_change().rolling(20).std()
    )
    
    print("  - 거래량 비율...")
    df['feature_volume_ratio'] = df.groupby('ticker')['volume'].transform(
        lambda x: calc_volume_ratio(x, 20)
    )
    
    print("  - RSI (14일)...")
    df['feature_rsi_14'] = df.groupby('ticker')['close'].transform(
        lambda x: calc_rsi(x, 14)
    )
    
    print("  - MACD...")
    # MACD는 그룹별로 계산 후 할당
    macd_results = []
    for ticker, group in df.groupby('ticker'):
        macd, macd_signal, macd_hist = calc_macd(group['close'])
        temp = pd.DataFrame({
            'ticker': ticker,
            'date': group['date'].values,
            'feature_macd': macd.values,
            'feature_macd_signal': macd_signal.values,
            'feature_macd_hist': macd_hist.values
        })
        macd_results.append(temp)
    
    macd_df = pd.concat(macd_results, ignore_index=True)
    df = df.merge(macd_df, on=['ticker', 'date'], how='left')
    
    print("  - Bollinger Bands...")
    bollinger_results = []
    for ticker, group in df.groupby('ticker'):
        upper, middle, lower = calc_bollinger(group['close'])
        temp = pd.DataFrame({
            'ticker': ticker,
            'date': group['date'].values,
            'feature_bollinger_upper': upper.values,
            'feature_bollinger_middle': middle.values,
            'feature_bollinger_lower': lower.values
        })
        bollinger_results.append(temp)
    
    bollinger_df = pd.concat(bollinger_results, ignore_index=True)
    df = df.merge(bollinger_df, on=['ticker', 'date'], how='left')
    
    feature_cols = [c for c in df.columns if c.startswith('feature_')]
    print(f"✅ {len(feature_cols)}개 Feature 생성 완료")
    
    return df

In [4]:
# ==========================================
# Target Builder: 예측 목표 생성
# ==========================================

def build_targets(df: pd.DataFrame, horizon: int = 5) -> pd.DataFrame:
    """
    Target 변수 생성
    
    책임
    ----
    - n일 후 수익률 계산
    - 로그 수익률 계산
    - 방향성 라벨 생성
    
    비책임
    ------
    - 정규화/스케일링 (03단계)
    - 샘플 제거 (03단계)
    
    Parameters
    ----------
    df : DataFrame
        Feature가 추가된 데이터
    horizon : int
        예측 기간 (일)
        
    Returns
    -------
    DataFrame
        원본 + target_* 컬럼 추가
    """
    print(f"\n🎯 Building Targets (Horizon: {horizon}일)...")
    
    df = df.copy()
    
    # 종목별 미래 가격 shift
    print("  - Forward return 계산...")
    df['future_close'] = df.groupby('ticker')['close'].shift(-horizon)
    
    # 수익률 계산
    df['target_return'] = (df['future_close'] / df['close']) - 1
    df['target_log_return'] = np.log(df['future_close'] / df['close'])
    
    # 방향성 라벨 (상승=1, 하락=0)
    df['target_direction'] = (df['target_return'] > 0).astype(int)
    
    # 임시 컬럼 제거
    df = df.drop(columns=['future_close'])
    
    # NaN 통계
    target_cols = [c for c in df.columns if c.startswith('target_')]
    nan_count = df[target_cols].isna().sum().sum()
    total_count = len(df) * len(target_cols)
    
    print(f"✅ {len(target_cols)}개 Target 생성 완료")
    print(f"   NaN: {nan_count}/{total_count} ({nan_count/total_count*100:.1f}%)")
    print(f"   (마지막 {horizon}일치 데이터는 Target이 없음 - 정상)")
    
    return df

In [5]:
# ==========================================
# Universe Meta Builder: 운영 판단용 메타 지표
# ==========================================

def build_universe_meta(df: pd.DataFrame) -> pd.DataFrame:
    """
    Universe 선정용 메타 지표 생성
    
    책임
    ----
    03단계(모델 학습/운영)에서 필요한 판단 지표를 사전 계산
    - 유동성 점수
    - 리스크 복합 점수
    - 거래 가능 여부 플래그
    
    목적
    ----
    "03단계가 도메인 판단을 하지 않도록" 미리 계산
    
    Parameters
    ----------
    df : DataFrame
        Feature + Target이 추가된 데이터
        
    Returns
    -------
    DataFrame
        원본 + 메타 지표 컬럼 추가
    """
    print("\n🏛️ Building Universe Meta...")
    
    df = df.copy()
    
    # 1. 유동성 점수 (거래대금 기반)
    print("  - 유동성 점수 계산...")
    if 'amount' in df.columns:
        # amount가 있으면 직접 사용
        df['liquidity_score'] = df['amount']
    else:
        # 없으면 close * volume으로 추정
        df['liquidity_score'] = df['close'] * df['volume']
    
    # 20일 평균 유동성
    df['liquidity_score'] = df.groupby('ticker')['liquidity_score'].transform(
        lambda x: x.rolling(20).mean()
    )
    
    # 2. 리스크 복합 점수 (단순 버전)
    print("  - 리스크 점수 계산...")
    # 변동성이 높을수록 리스크 증가
    df['risk_volatility'] = df['feature_volatility_20'].fillna(0)
    
    # 거래량 급증도 리스크 (테마주 의심)
    df['risk_volume_surge'] = (df['feature_volume_ratio'] > 3.0).astype(int)
    
    # 복합 리스크 (0~1 정규화)
    df['risk_composite'] = (
        df['risk_volatility'] / df['risk_volatility'].max() * 0.5 +
        df['risk_volume_surge'] * 0.5
    )
    
    # 3. 거래 가능 여부 플래그 (Placeholder)
    # 실제로는 별도 API 조회 필요 (get_market_trading_halt 등)
    # 현재는 유동성 극단적 저하를 대리 지표로 사용
    print("  - 거래 가능 여부 플래그...")
    df['is_suspended'] = (df['volume'] == 0).astype(int)
    df['is_delisted'] = 0  # Placeholder (실제로는 별도 관리 필요)
    
    meta_cols = ['liquidity_score', 'risk_composite', 'is_suspended', 'is_delisted']
    print(f"✅ {len(meta_cols)}개 메타 지표 생성 완료")
    
    return df

In [6]:
# ==========================================
# Schema Validation: 필수 컬럼 검증
# ==========================================

def validate_schema(df: pd.DataFrame) -> bool:
    """
    최종 데이터셋 스키마 검증
    
    필수 컬럼 그룹
    ------------
    1. 기본: date, ticker
    2. Feature: feature_*
    3. Target: target_*
    4. Meta: liquidity_score, risk_composite, is_suspended, is_delisted
    
    Returns
    -------
    bool
        검증 통과 여부
    """
    print("\n✔️  Schema Validation...")
    
    # 1. 기본 컬럼
    required_base = ['date', 'ticker']
    missing_base = [c for c in required_base if c not in df.columns]
    
    if missing_base:
        print(f"❌ 필수 기본 컬럼 누락: {missing_base}")
        return False
    print(f"   ✅ 기본 컬럼: {required_base}")
    
    # 2. Feature 컬럼
    feature_cols = [c for c in df.columns if c.startswith('feature_')]
    if len(feature_cols) == 0:
        print("❌ Feature 컬럼이 하나도 없습니다")
        return False
    print(f"   ✅ Feature: {len(feature_cols)}개")
    
    # 3. Meta 컬럼 (Target은 03단계에서 생성하므로 제외)
    required_meta = ['liquidity_score', 'risk_composite', 'is_suspended', 'is_delisted']
    missing_meta = [c for c in required_meta if c not in df.columns]
    
    if missing_meta:
        print(f"⚠️  권장 메타 컬럼 누락: {missing_meta}")
        print("   (경고만 표시, 저장은 진행)")
    else:
        print(f"   ✅ 메타: {required_meta}")
    
    # 4. 데이터 통계
    print(f"\n📊 최종 데이터 통계:")
    print(f"   - 행 수: {len(df):,}")
    print(f"   - 종목 수: {df['ticker'].nunique():,}")
    print(f"   - 기간: {df['date'].min()} ~ {df['date'].max()}")
    print(f"   - 전체 컬럼 수: {len(df.columns)}")
    
    # 5. NaN 체크 (Feature에 NaN이 있으면 경고)
    feature_nan = df[feature_cols].isna().sum().sum()
    if feature_nan > 0:
        print(f"\n⚠️  Feature NaN 발견: {feature_nan}개")
        print("   → 거래정지 등 의미 있는 NaN일 수 있음 (정상)")
    
    return True

## 🚀 Main Pipeline

In [7]:
# ==========================================
# 1. Load Raw Data
# ==========================================

print("📥 Loading Raw CSV Files...")

csv_files = list(RAW_DATA_DIR.glob("*.csv"))
print(f"   발견된 파일: {len(csv_files)}개")

if not csv_files:
    raise FileNotFoundError(f"Raw 데이터가 없습니다: {RAW_DATA_DIR}")

# 모든 CSV 로드
all_data = []
for csv_file in tqdm(csv_files, desc="Loading CSVs"):
    try:
        df_temp = pd.read_csv(csv_file)
        df_temp['date'] = pd.to_datetime(df_temp['date'])
        all_data.append(df_temp)
    except Exception as e:
        print(f"⚠️  {csv_file.name} 로드 실패: {e}")

# 통합
df_raw = pd.concat(all_data, ignore_index=True)
df_raw = df_raw.sort_values(['ticker', 'date']).reset_index(drop=True)

print(f"\n✅ Raw 데이터 로드 완료")
print(f"   - 총 행 수: {len(df_raw):,}")
print(f"   - 종목 수: {df_raw['ticker'].nunique():,}")
print(f"   - 기간: {df_raw['date'].min()} ~ {df_raw['date'].max()}")
print(f"\n   컬럼: {list(df_raw.columns)}")

📥 Loading Raw CSV Files...
   발견된 파일: 2905개


Loading CSVs: 100%|███████████████████████████████████████████████████████████████| 2905/2905 [00:07<00:00, 390.94it/s]



✅ Raw 데이터 로드 완료
   - 총 행 수: 681,346
   - 종목 수: 2,905
   - 기간: 2025-01-02 00:00:00 ~ 2025-12-26 00:00:00

   컬럼: ['date', 'ticker', 'open', 'high', 'low', 'close', 'volume', 'change_pct']


In [8]:
# ==========================================
# 2. Build Features
# ==========================================

df_features = build_features(df_raw)


🔨 Building Features...
  - 이동평균 (MA 5, 20, 60)...
  - 변동성 (20일 표준편차)...
  - 거래량 비율...
  - RSI (14일)...
  - MACD...
  - Bollinger Bands...
✅ 12개 Feature 생성 완료


In [9]:
# ==========================================
# 3. History Filtering
# ==========================================

# 종목별로 60일 이후 데이터만 유지
def filter_by_history(group, min_history=60):
    return group.iloc[MIN_HISTORY:] if len(group) > MIN_HISTORY else pd.DataFrame()

df_filtered = filter_by_history(df_features, min_history=MIN_HISTORY)

In [10]:
# ==========================================
# 4. Build Universe Meta
# ==========================================

df_final = build_universe_meta(df_filtered)


🏛️ Building Universe Meta...
  - 유동성 점수 계산...
  - 리스크 점수 계산...
  - 거래 가능 여부 플래그...
✅ 4개 메타 지표 생성 완료


In [11]:
# ==========================================
# 5. Validate Schema
# ==========================================

if not validate_schema(df_final):
    raise ValueError("스키마 검증 실패! 위의 에러 메시지를 확인하세요.")


✔️  Schema Validation...
   ✅ 기본 컬럼: ['date', 'ticker']
   ✅ Feature: 12개
   ✅ 메타: ['liquidity_score', 'risk_composite', 'is_suspended', 'is_delisted']

📊 최종 데이터 통계:
   - 행 수: 681,286
   - 종목 수: 2,905
   - 기간: 2025-01-02 00:00:00 ~ 2025-12-26 00:00:00
   - 전체 컬럼 수: 26

⚠️  Feature NaN 발견: 551509개
   → 거래정지 등 의미 있는 NaN일 수 있음 (정상)


In [12]:
# ==========================================
# 6. Save to Parquet
# ==========================================

print(f"\n💾 Saving to Parquet...")
print(f"   경로: {OUTPUT_FILE}")

# ticker를 명시적으로 문자열로 변환 (Parquet 타입 추론 오류 방지)
df_final['ticker'] = df_final['ticker'].astype(str)

df_final.to_parquet(
    OUTPUT_FILE,
    compression='snappy',
    index=False
)

# 저장 확인
file_size_mb = OUTPUT_FILE.stat().st_size / (1024 * 1024)
print(f"✅ 저장 완료!")
print(f"   - 파일 크기: {file_size_mb:.2f} MB")
print(f"   - 행 수: {len(df_final):,}")
print(f"   - 컬럼 수: {len(df_final.columns)}")


💾 Saving to Parquet...
   경로: data\03_processed\dataset_20251226.parquet
✅ 저장 완료!
   - 파일 크기: 72.41 MB
   - 행 수: 681,286
   - 컬럼 수: 26


## ✅ 결과 검증

In [13]:
# 저장된 파일 다시 로드하여 검증
df_verify = pd.read_parquet(OUTPUT_FILE)

print("🔍 저장된 데이터 검증:")
print(f"\n📋 컬럼 그룹별 개수:")
print(f"   - 기본: {len([c for c in df_verify.columns if not c.startswith('feature_')])}")
print(f"   - Feature: {len([c for c in df_verify.columns if c.startswith('feature_')])}")

print(f"\n📊 샘플 데이터 (처음 3행):")
display(df_verify.head(3))

print(f"\n🔢 Feature NaN 체크:")
feature_cols = [c for c in df_verify.columns if c.startswith('feature_')]
nan_count = df_verify[feature_cols].isna().sum()
if nan_count.sum() > 0:
    print("   NaN이 있는 Feature:")
    print(nan_count[nan_count > 0])
    print("   → 거래정지 등 의미 있는 NaN일 수 있음")
else:
    print("   ✅ 모든 Feature가 완전함 (NaN 없음)")

🔍 저장된 데이터 검증:

📋 컬럼 그룹별 개수:
   - 기본: 14
   - Feature: 12

📊 샘플 데이터 (처음 3행):


,date,ticker,open,high,low,close,volume,change_pct,feature_ma_5,feature_ma_20,...,feature_macd_hist,feature_bollinger_upper,feature_bollinger_middle,feature_bollinger_lower,liquidity_score,risk_volatility,risk_volume_surge,risk_composite,is_suspended,is_delisted
0,2025-04-03,20,5980,6080,5910,6010,27400,0.166667,6020.0,6137.0,...,-15.823171,6303.461628,6137.0,5970.538372,NaN,0.006555,0,0.000352,0,0
1,2025-04-04,20,6050,6100,5910,6010,66624,0.000000,6010.0,6123.0,...,-13.976443,6282.221792,6123.0,5963.778208,NaN,0.006574,0,0.000353,0,0
2,2025-04-07,20,5990,5990,5740,5750,73155,-4.326123,5962.0,6098.5,...,-28.312267,6320.384275,6098.5,5876.615725,NaN,0.011258,0,0.000605,0,0



🔢 Feature NaN 체크:
   NaN이 있는 Feature:
feature_ma_5                 11613
feature_ma_20                54950
feature_ma_60               169667
feature_volatility_20        57832
feature_volume_ratio         54950
feature_rsi_14               37647
feature_bollinger_upper      54950
feature_bollinger_middle     54950
feature_bollinger_lower      54950
dtype: int64
   → 거래정지 등 의미 있는 NaN일 수 있음


## ✅ 완료!

### 생성된 파일
- `data/03_processed/dataset.parquet`

### 이 데이터셋의 특징
1. **Feature 준비 기간 제거**: 각 종목의 처음 60일은 제외됨
2. **완전한 Feature**: 모든 행이 계산 가능한 Feature를 가짐
3. **의미 있는 NaN 보존**: 거래정지 등으로 인한 NaN은 유지됨
4. **Target 없음**: 03단계에서 필요 시 생성

### 다음 단계
```bash
python scripts/03_train_predict.py
```

### 03단계에서 할 일
1. **Target 생성**: 예측 목표 정의 후 생성
2. **Train/Test Split**: Walk-Forward 시계열 분할
3. **Feature Scaling**: RobustScaler 적용
4. **모델 학습**: LightGBM 훈련
5. **Universe 선정**: Meta 지표 활용

### 데이터 사용 예시
```python
df = pd.read_parquet('data/03_processed/dataset.parquet')

# Target 생성 (03단계)
df['target'] = df.groupby('ticker')['close'].shift(-5) / df['close'] - 1

# Feature 추출
features = [c for c in df.columns if c.startswith('feature_')]
X = df[features]
y = df['target']
```